# SOAP analysis of Si trajectory

In [51]:
from ase.io import read, write
import os
import pandas as pd
import numpy as np
from quippy.descriptors import Descriptor
from ase.build import bulk

In [2]:
from os.path import join

In [7]:
Si_structures = read('structures/Si_10katom_simulation.xyz', ':')
print(Si_structures[0])

In [45]:
def read_lammps_log(file):
    '''Reads LAMMPS log thermo data into a pandas DataFrame
    Params: filename
    Returns: pandas DataFrame'''

    with open(file, 'r') as f:
        out = f.readlines()
    flag = False
    first_time = True
    for i, val in enumerate(out):
        test = val.split()
        test.append('')
        if test[0] == 'Step':
            if first_time:
                dat = [[] for j in range(len(out[i+1].split()))]
                dat_head = val.split()
                first_time = False
            flag = True
            continue

        if flag:
            try:
                for j, num in enumerate(val.split()):
                    dat[j].append(float(num))
            except:
                flag = 0
    dat = np.array(dat) # turn into a DataFrame with header
    df = pd.DataFrame(dat[:].T, columns=dat_head[:])
    df.drop_duplicates(subset=df.columns[-1], inplace=True)
    
    return df

In [49]:
thermo = read_lammps_log('structures/log_npt_Si_mtp.dat')#
thermo.head()

,Step,CPU,Temp,f_TempAve,Press,f_PressAve,f_PEAve_Atom,Volume,f_vAve,c_MSD[4]
0,0.0,0.000000,500.00000,0.00000,200.47885,0.000000,0.00000,206036.57,0.00,0.000000
2,1000.0,66.406337,499.06923,496.43517,469.56218,-0.746819,-162.92661,205947.39,206035.67,0.099112
3,2000.0,132.756830,504.12737,500.23940,367.79051,-41.270840,-162.92650,205852.77,206019.93,0.104916
4,3000.0,199.107770,498.82079,500.74544,108.73607,-61.930008,-162.92650,206026.93,206014.29,0.102298
5,4000.0,265.756330,499.03487,499.13229,-185.78896,-85.978728,-162.92618,206067.84,206029.74,0.103973


# Set up SOAP descriptor

In [ ]:
soap = Descriptor('soap cutoff=3.5 cutoff_transition_width=0.5 n_max=8 l_max=6 atom_sigma=0.5 n_Z=1 Z={14} n_species=1 species_Z={{14}}')

# Create reference crystal structures